In [3]:
import sys
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Get the absolute path of the project root (modify if needed)
project_root = os.path.abspath(r"C:\Users\wmate\OneDrive\UNFC\Term 2\Predictive Analytics\Heart_Disease_Risk_Prediction")

# Add the project root to sys.path if not already added
if project_root not in sys.path:
    sys.path.append(project_root)

# Importar el módulo
from src import visualizations

# Recargar el módulo después de modificarlo
importlib.reload(visualizations)

# Ahora puedes volver a usar las funciones actualizadas
from src.visualizations import plot_distribution_by_category,plot_correlation_by_category
from src.tools import categorizar_tipos_basicos, categorizar_tipos_detallados, crosstab_percentages

In [5]:
import pickle

class CustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        # Redirect module if it is the old numpy internal module
        if module == "numpy._core.numeric":
            module = "numpy.core.numeric"
        return super().find_class(module, name)

# Use the custom unpickler to load your pickle file
with open('../data/processed/df_loan.pkl', 'rb') as f:
    df = CustomUnpickler(f).load()

# Take a sample as before
df = df.sample(frac=0.1, random_state=42)
df

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
75255,75256,4260981,47,18,single,rented,no,Politician,Hindupur,Andhra_Pradesh,7,13,Low Risk
192435,192436,8529345,44,2,single,rented,no,Computer_hardware_engineer,Narasaraopet,Andhra_Pradesh,2,10,Low Risk
154839,154840,7848654,55,9,single,rented,no,Software_Developer,Patna,Bihar,9,13,Low Risk
59774,59775,8491491,61,20,single,rented,no,Comedian,Haldia,West_Bengal,8,11,Low Risk
63788,63789,1537266,78,13,single,rented,no,Web_designer,Coimbatore,Tamil_Nadu,9,12,Low Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213483,213484,455805,72,0,single,rented,no,Aviator,Raurkela_Industrial_Township,Odisha,0,12,High Risk
20987,20988,754014,69,2,single,rented,no,Software_Developer,Deoghar,Jharkhand,2,13,Low Risk
91768,91769,3666346,56,12,single,rented,no,Politician,Bhusawal,Maharashtra,12,11,Low Risk
221315,221316,3543581,41,9,single,rented,no,Hotel_Manager,Kurnool[18],Andhra_Pradesh,9,14,High Risk


In [6]:
df.dtypes

Id                      int64
Income                  int64
Age                     int64
Experience              int64
Married/Single       category
House_Ownership      category
Car_Ownership        category
Profession           category
CITY                 category
STATE                category
CURRENT_JOB_YRS         int64
CURRENT_HOUSE_YRS       int64
Risk_Flag            category
dtype: object

In [7]:
df.describe()

,Id,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS
count,25200.000000,2.520000e+04,25200.000000,25200.000000,25200.000000,25200.000000
mean,125928.411984,4.984786e+06,49.665714,10.105516,6.370476,11.991786
std,72847.638393,2.863783e+06,16.971351,6.012166,3.653866,1.401718
min,13.000000,1.031000e+04,21.000000,0.000000,0.000000,10.000000
25%,62714.500000,2.530244e+06,35.000000,5.000000,4.000000,11.000000
50%,125575.000000,4.973814e+06,49.000000,10.000000,6.000000,12.000000
75%,189055.500000,7.444228e+06,64.000000,15.000000,9.000000,13.000000
max,251992.000000,9.999180e+06,79.000000,20.000000,14.000000,14.000000


In [8]:
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df_sample = df.sample(frac=0.1, random_state=42)

# Separate features (X) and target (y)
X = df_sample.drop('Risk_Flag', axis=1)
y = df_sample['Risk_Flag']

# Handle categorical features using Label Encoding
categorical_cols = X.select_dtypes(include=['category']).columns
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 31/32 [00:04<00:00,  9.24it/s]

[LightGBM] [Info] Number of positive: 1779, number of negative: 237
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000512 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 955
[LightGBM] [Info] Number of data points in the train set: 2016, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.882440 -> initscore=2.015747
[LightGBM] [Info] Start training from score 2.015747


100%|██████████| 32/32 [00:04<00:00,  6.41it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
Perceptron,0.74,0.58,None,0.77,0.04
NearestCentroid,0.59,0.56,None,0.65,0.04
BaggingClassifier,0.87,0.56,None,0.84,0.21
DecisionTreeClassifier,0.79,0.56,None,0.79,0.05
ExtraTreesClassifier,0.88,0.56,None,0.84,0.48
LabelPropagation,0.80,0.55,None,0.80,0.28
LabelSpreading,0.80,0.55,None,0.80,0.38
LGBMClassifier,0.88,0.55,None,0.84,0.91
RandomForestClassifier,0.88,0.55,None,0.84,0.77


In [9]:
path_Reults_model="./data/models_results"
models.sort_values('F1 Score', ascending=False).to_csv(os.path.join(project_root, path_Reults_model, 'models_results v2.csv'))